<a href="https://colab.research.google.com/github/HarshitKumar-pixel/BEST-AI-WORK/blob/main/karpathy1.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [64]:
import math
import numpy as np
import matplotlib.pyplot as plt
import random
%matplotlib inline

In [65]:
class Value:
  def __init__(self,data, _children=() , _op='',label='') :
    self.data=data
    self.grad = 0.0
    self._backward = lambda : None
    self._prev = set(_children)
    self._op = _op
    self.label = label

  def __repr__(self):
    return f"Value(data={self.data})"

  def __add__(self,other):
    other = other if isinstance(other,Value) else Value(other)
    out = Value(self.data + other.data,(self,other),'+')

    def backward():
      self.grad += 1.0*out.grad
      other.grad += 1.0*out.grad
    out._backward = backward

    return out

  def __mul__(self,other):
    other = other if isinstance(other,Value) else Value(other)
    out = Value(self.data * other.data,(self,other),'*')
    def backward():
      self.grad += other.data * out.grad
      other.grad += self.data * out.grad

    out._backward = backward
    return out

  def __pow__(self, other):
    assert isinstance(other, (int, float)), "only supporting int/float powers for now"
    out = Value(self.data ** other, (self,), f'**{other}')

    def backward():
        self.grad += other * (self.data ** (other - 1)) * out.grad
    out._backward = backward

    return out

  def __neg__(self):
    return self*-1

  def __sub__(self,other):
    return self + (-other)

  def __rmul__(self,other):
    return self*other

  def __truediv__(self,other):
    return self * other**-1

  def tanh(self):
    x=self.data
    t = (math.exp(2*x)-1)/(math.exp(2*x)+1)
    out = Value(t,(self,),'tanh')
    def backward():
      self.grad += (1 - out.data**2)*out.grad

    out._backward = backward
    return out

  def exp(self):
    x = self.data
    out = Value(math.exp(x),(self, ),_op='exp')

    def backward():
      self.grad+=out.data*out.grad

    out._backward=backward

    return out

  def backward(self):
    topo = []
    visited = set()
    def build_topo(v):
      if v not in visited:
        visited.add(v)
        for child in v._prev:
          build_topo(child)
        topo.append(v)

    build_topo(self)

    self.grad = 1.0

    for node in reversed(topo):
      node._backward()




In [66]:
from graphviz import Digraph

def trace(root):
    # builds a set of all nodes and edges in a graph
    nodes, edges = set(), set()
    def build(v):
        if v not in nodes:
            nodes.add(v)
            for child in v._prev:
                edges.add((child, v))
                build(child)
    build(root)
    return nodes, edges

def draw_dot(root):
    dot = Digraph(format='svg', graph_attr={'rankdir': 'LR'})  # LR = left to right

    nodes, edges = trace(root)
    for n in nodes:
        uid = str(id(n))
        # for any value in the graph, create a rectangular ('record') node for it
        dot.node(
            name=uid,
            label="{ %s | data %.4f | grad %.4f }" % (n.label, n.data, n.grad),
            shape='record'
        )
        if n._op:
            # if this value is a result of some operation, create an op node for it
            dot.node(name=uid + n._op, label=n._op)
            # and connect this node to it
            dot.edge(uid + n._op, uid)

    for n1, n2 in edges:
        # connect n1 to the op node of n2
        dot.edge(str(id(n1)), str(id(n2)) + n2._op)

    return dot


In [221]:

class  Neuron:
  def __init__(self,nin):
    self.w = [Value(random.uniform(-1,1)) for i in range(nin)]
    self.b = Value(random.uniform(-1,1))

  def __call__(self,x):
    act = sum((wi*xi for wi,xi in zip(self.w,x)),self.b)
    out = act.tanh()
    return out

  def parameters(self):
    return self.w + [self.b]

class Layer:
  def __init__(self,nin,nout):
    self.neurons = [Neuron(nin) for _ in range(nout)]

  def __call__(self,x):
    out = [n(x) for n in self.neurons]
    return out[0] if len(out) == 1 else out

  def parameters(self):
    params = []
    for neuron in self.neurons:
      sp  = neuron.parameters()
      params.extend(sp)
    return params

class MLP:
  def __init__(self, nin, nouts):
    temp = [nin] + nouts
    self.layers = [Layer(temp[i],temp[i+1]) for i in range(len(nouts))]

  def __call__(self,x):
    for layer in self.layers:
      x = layer(x)

    return x

  def parameters(self):
    params = []
    for layer in self.layers:
      sp = layer.parameters()
      params.extend(sp)
    return params


In [298]:
# change the values from here

n = MLP(3,[10,10,1])

In [299]:
# this is a tiny data set and its outputs

xs = [
    [2.0, 3.0, -1.0],
    [3.0, -1.0, 0.5],
    [0.5, 1.0, 1.0],
    [1.0, 1.0, -1.0],
]
ys = [-1.0,1.0,1.0,-1.0]
parameters = n.parameters()

In [305]:
# this is the optimization step . can change the values

for i in range(1000):
  # forward pass
  ypred = [n(x) for x in xs]
  loss = sum(((ypredt - yst)**2 for ypredt,yst in zip(ypred , ys)),Value(0.0))

  # zero grading: if we dont do this step the derivatives will keep accumulating like dy/dx + dy2/dx2 + ....... and so on
  for p in parameters:
    p.grad = 0.0

  # the backard pass assigning gradients to parameters at that particular point to minimize the loss
  loss.backward()

  # this is the gradient descent which is shifting the weights in the opposite direction of their gradient w.r.t loss so it decreases the loss
  for p in parameters:
    p.data+= -0.1*p.grad


loss

Value(data=4.216650543257885e-05)

In [304]:
ypred

[Value(data=-0.997558178589164),
 Value(data=0.9989392753294727),
 Value(data=0.9945173358110285),
 Value(data=-0.9948017128160901)]